# <center>Machine Learning: Assignment 2</center>
**<center>郑源泽  19307130077</center>**

## 任务描述
设计一个回归系统来预测住房价格。

数据集：https://www.kaggle.com/vikrishnan/boston-house-prices（也可在Sklearn上找到）。

回归算法应该包含 ridge regression 和 lasso regression. 

## 数据集描述
数据集来自于1970年的波士顿标准都市统计区（SMSA）。
其各项数据的详细意义在kaggle数据集的描述中有详细说明，这里不再赘述。
我们需要预测的是房屋的价格，即MEDV。

数据集的大概情况如下：

In [189]:
from sklearn.datasets import load_boston
import sklearn.preprocessing as pre
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, RidgeCV, Ridge, LassoCV
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import numpy as np

boston = load_boston()
x, y = boston.data, boston.target
print("data shape:"+str(x.shape))
print("target shape:"+str(y.shape))
print("dataset size:"+str(len(x)))
print("vector size:"+str(len(x[0])))
# print(type(x[0][0]), type(y[0]))
# print(x[0])
# print(y[0])

data shape:(506, 13)
target shape:(506,)
dataset size:506
vector size:13


## 算法描述
常见的三种线性回归
1. simple linear regression
2. ridge regression
3. lasso regression

进⾏线性回归的计算其实有两种⽅法：最⼩⼆乘法和梯度下降法。本次实验调⽤sklearn库的线性模型，通过查阅⽂档，他们其实都是基于最⼩⼆乘法来实现的。因此模型描述部分介绍最⼩⼆乘法求解线性回归的原理。

令$x_j^{(i)}$表示数据集第$i$个数据的第$j$个属性取值，$y^{(i)}$表示数据集第$i$个数据的标签值，$z^{(i)}$表示第$i$组数据的计算预测值。数据集一共有$m$个数据，$n$种属性。

矩阵形式表示，$x_i=\begin{bmatrix}1 & x_1^{(i)} & \cdots & x_n^{(i)}\end{bmatrix}$，$X=\begin{bmatrix}1 & x_1^{(1)} & \cdots & x_n^{(1)} \\ 1 & x_1^{(2)} & \cdots & x_n^{(2)} \\ \cdots & \cdots & \cdots & \cdots \\ 1 & x_1^{(m)} & \cdots & x_n^{(m)}\end{bmatrix}$，参数$W=\begin{bmatrix}w_0 \\ w_1 \\ \vdots \\ w_n\end{bmatrix}$，数据集标签为$y=\begin{bmatrix}y^{(1)} \\ y^{(2)} \\ \vdots \\ y^{(m)}\end{bmatrix}$

模型定义为$f(x)=\sum_{i=0}^n{w_ix_i}$，其中$x_0=1$，矩阵表示为$f(x)=XW$。我们的目的就是寻找最合适的$W$来最好效果地拟合和预测。

线性回归试图学得$z = \sum_{i=0}^nw_ix_i$，使得$z\simeq y$。而回归任务中常用的学习$W$手段便是均方差(MSE-mean squared error)：
$$J=\frac{1}{2m}\sum_{i=1}^m(z_i-y_i)^2=\frac{1}{2m}\sum_{i=1}^m(y^{(i)}-\sum_{j=0}^nw_jx_j^{(i)})^2$$
我们要让均方差最小，也就要使得$\frac{\partial J}{\partial w}$为0。以$w_k$为例
$$\frac{\partial J}{\partial w_k}=\frac{\partial( \frac{1}{2m}\sum_{i=1}^m(y^{(i)}-\sum_{j=0}^nw_jx_j^{(i)})^2)}{\partial w_k}=\frac{1}{m}\sum_{i=1}^m[(y^{(i)}-\sum_{j=0}^nw_jx_j^{(i)})(-x_k^{(i)})]=0$$

由此可以得到一个关于$w_k$的方程组，可由此解出各个$w_k$的值，方程组如下:
$$\begin{cases}
\sum_{i=1}^m[(y^{(i)}-\sum_{j=0}^nw_jx_j^{(i)})(-x_0^{(i)})]=0\\
\sum_{i=1}^m[(y^{(i)}-\sum_{j=0}^nw_jx_j^{(i)})(-x_1^{(i)})]=0\\
\cdots\\
\sum_{i=1}^m[(y^{(i)}-\sum_{j=0}^nw_jx_j^{(i)})(-x_n^{(i)})]=0
\end{cases}$$
化简如下:
$$\begin{cases}
\sum_{j=0}^n[w_jx_0^{(i)}(\sum_{i=1}^mx_j^{(i)})]=\sum_{i=1}^mx_0^{(i)}y^{(i)}\\
\sum_{j=0}^n[w_jx_1^{(i)}(\sum_{i=1}^mx_j^{(i)})]=\sum_{i=1}^mx_1^{(i)}y^{(i)}\\
\cdots\\
\sum_{j=0}^n[w_jx_n^{(i)}(\sum_{i=1}^mx_j^{(i)})]=\sum_{i=1}^mx_n^{(i)}y^{(i)}
\end{cases}$$

我们要求解向量$W=\begin{bmatrix}w_0 \\ w_1 \\ \vdots \\ w_n\end{bmatrix}$。上方程组用矩阵形式表示如下

$$AW=B \\ A^{T}AW=A^{T}B \\ W=(A^TA)^{-1}A^TB$$

RidgeRegression(岭回归)

为最小二乘法估计中加入一个扰动$\lambda I$，使得原先无法求出广义逆的情况变成可以求出其广义逆，使得问题稳定并得以求解。同时，岭回归在计算`loss`时加入$L2$正则化，即:
$$J = \frac{1}{2m}\sum_{i=1}^m(z^{(i)}-y^{(i)})^2 + \lambda\sum_{i=0}^nw_i^2$$

LassoRegression(Lasso回归)

Lasso回归与岭回归类似，同样引入扰动$\lambda I$，但在计算`loss`时加上$L1$正则化，也即:
$$J = \frac{1}{2m}\sum_{i=1}^m(z^{(i)}-y^{(i)})^2 + \lambda\sum_{i=0}^n|w_i|$$

## 算法实现

In [190]:
from sklearn.model_selection import train_test_split
boston = load_boston()

x, y = boston.data, boston.target
# x = pre.StandardScaler().fit_transform(x)
x = pre.MinMaxScaler().fit_transform(x)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1930)


从两个维度来判断模型拟合的好坏
1. 拟合函数的R2,⽤model.score函数来获取，体现拟合曲线对于样本点的拟合
2. 预测房价值和真实房价值的差的绝对值平均，⽤mean_absolute_percentage_error函数获得，可以体现预测的是否准确

- 简单线性回归

In [191]:
def simple_linear_regression(x_train, x_test, y_train, y_test):
    model = LinearRegression()
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    # print("mean_absolute_percentage_error:", round(mean_absolute_percentage_error(y_test, y_pred),4))
    # print("score:", round(model.score(x_train, y_train),4))
    # print("\n")
    return round(model.score(x_train, y_train),4), round(mean_absolute_percentage_error(y_test, y_pred),4)

- Ridge回归

In [192]:
def ridge_regression(x_train, x_test, y_train, y_test, my_alpha = 0.1):
    model = RidgeCV(alphas = [0.000001, 0.000003, 0.00001, 0.00003, 0.0001, 0.0003,
                            0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30, 100])
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print("最佳alpha是：", model.alpha_)
    # print("这是ridge_regression的测试结果：")
    # print("mean_absolute_percentage_error:", round(mean_absolute_percentage_error(y_test, y_pred), 4))
    # print("score:", round(model.score(x_train, y_train), 4))
    # print("\n")
    return round(model.score(x_train, y_train),4), round(mean_absolute_percentage_error(y_test, y_pred),4)

- Lasso回归

In [193]:
def lasso_regression(x_train, x_test, y_train, y_test, my_alpha = 0.1):
    model = LassoCV(alphas = [0.0000000000001, 0.0000000000003, 0.0000001, 0.0000003, 0.000001, 0.000003,
                              0.00001, 0.00003, 0.0001, 0.0003, 0.001, 0.003, 0.01, 0.03, 0.1,
                              0.3, 1, 3, 10, 30, 100])
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print("最佳alpha是：", model.alpha_)
    #print("这是lasso_regression的测试结果：")
    #print("mean_absolute_percentage_error:", round(mean_absolute_percentage_error(y_test, y_pred), 4))
    #print("score:", round(model.score(x_train, y_train), 4))
    #print("\n")
    return round(model.score(x_train, y_train),4), round(mean_absolute_percentage_error(y_test, y_pred),4)

In [194]:
err = 0
score = 0
for i in range(1024, 1034):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i+5)
    # t1, t2 = simple_linear_regression(x_train, x_test, y_train, y_test)
    # t1, t2 = ridge_regression(x_train, x_test, y_train, y_test)
    t1, t2 = lasso_regression(x_train, x_test, y_train, y_test)
    err = err + t2
    score = score + t1
print("average mean_absolute_percentage_error:", err/10)
print("average score:", score/10)

最佳alpha是： 1e-13
最佳alpha是： 0.003
最佳alpha是： 3e-13
最佳alpha是： 0.003
最佳alpha是： 1e-13
最佳alpha是： 1e-13
最佳alpha是： 0.003
最佳alpha是： 0.003
最佳alpha是： 1e-13
最佳alpha是： 1e-13
average mean_absolute_percentage_error: 0.17192999999999997
average score: 0.7415099999999999


## 实验结果
- 简单线性回归：0.7415
- Ridge回归：0.7411
- Lasso回归：0.7415
  
可能是数据集较规范，没有出现过拟合现象。岭回归和Lasso回归的结果与最简单的线性回归score结果相差不大。

## 总结
在本次实验中学习了线性回归的数学原理（最小二乘法），并利用一些现有的机器学习库与波士顿房价数据集进行了三种线性回归的手动实践。在实验中加深了对线性回归算法的理解，同时也对机器学习库的使用有了更深的认识。